In [1]:
import os, sys
sys.path.append("/dors/capra_lab/users/fongsl/tools/py_/")
import config_readwrite as crw
import split_filename

In [14]:
tissues = ["EBV", "Whole_Blood", "Spleen"]

name = "/data/hodges_lab/ATAC-STARR_B-cells/bin_human-evolution/config"
config, configfile_name = crw.read_config(name)

EQTL = config["EQTL"]["src_data_uniq"]

In [3]:
def write_tissue(file, line):
    with open(file, "a") as eqtl_writer:
        eqtl_writer.write(line)
        eqtl_writer.close()


In [4]:
val = 0
with open(EQTL, "r") as eqtlF:
    lines = eqtlF.readlines()
    for i, line in enumerate(lines):
        if i ==0:
            header = line
        else:
            for t in tissues:
                if t in line:
                    t_file = config["EQTL"][t.lower()]
                    break
                    #write_tissue(t_file, line)
                    

KeyboardInterrupt: 

In [15]:
for t in tissues:
    file = config["EQTL"][t.lower()]
    df = pd.read_csv(file, sep = '\t', header = None)
    df.columns = list(header.split("\t")) # add header
    
    df.to_csv(file, sep = '\t', index = False)

In [10]:
import pandas as pd
import dask.dataframe as dd

In [17]:
df = dd.read_csv(EQTL, sep = '\t', header=None)

In [18]:
df.head()

,0,1,2,3,4
0,chr10,100000234,100000235,C/T,Adipose_Subcutaneous
1,chr10,100000234,100000235,C/T,Adipose_Visceral_Omentum
2,chr10,100000234,100000235,C/T,Adrenal_Gland
3,chr10,100000234,100000235,C/T,Artery_Aorta
4,chr10,100000234,100000235,C/T,Artery_Coronary


In [30]:
df.loc[df[1]=="start"]

,0,1,2,3,4
npartitions=28,,,,,
,object,int64,int64,object,object
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [28]:
df = df.loc[df[1]!="start"]

In [29]:
df.groupby([0,1,2,3])[4].count().compute()

/home/fongsl/.conda/envs/sfenv/lib/python3.6/site-packages/dask/core.py:121: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*(_execute_task(a, cache) for a in args))


ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+--------+--------+----------+
| Column | Found  | Expected |
+--------+--------+----------+
| 1      | object | int64    |
| 2      | object | int64    |
+--------+--------+----------+

The following columns also raised exceptions on conversion:

- 1
  ValueError("invalid literal for int() with base 10: 'start'",)
- 2
  ValueError("invalid literal for int() with base 10: 'end'",)

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={1: 'object',
       2: 'object'}

to the call to `read_csv`/`read_table`.